# 040-rede2018_rede_gephi

In [1]:
ano_eleicao = '2018'

In [2]:
dbschema = f'rede{ano_eleicao}'
table_edges = f"{dbschema}.gephi_edges_{ano_eleicao}"
table_nodes = f"{dbschema}.gephi_nodes_{ano_eleicao}"
table_receitas = f"{dbschema}.receitas_{ano_eleicao}"
table_candidaturas = f"{dbschema}.candidaturas_{ano_eleicao}"

In [3]:
import sys
sys.path.append('../')
import mod_tse as mtse

In [4]:
import os
home = os.environ["HOME"]
local_dir = f'{home}/DADOS_TSE'

## CRIA A TABELA DE ARESTAS

In [5]:
query_create_table_edges = f"""
drop table if exists {table_edges};
create table {table_edges}
(
  Source character varying,
  Target character varying,
  Type character varying,
  Weight numeric(18,2),
  doador_tipo varchar,
  doador_uf varchar,
  doador_partido varchar,
  doador_cargo varchar,
  doador_genero varchar,
  doador_label varchar,
  receita_origem varchar,
  receptor_tipo varchar,
  receptor_uf varchar,
  receptor_partido varchar,
  receptor_cargo   varchar,
  receptor_genero  varchar,
  receptor_label character varying
);

CREATE INDEX ON {table_edges} (Source);
CREATE INDEX ON {table_edges} (Target);


insert into {table_edges}
SELECT
  doador_id           as Source ,
  receptor_id         as Target,
  'undirected'        as Type,
  sum(receita_valor)  as Weight,
  doador_tipo_cd      as  doador_tipo,
  doador_uf           as  doador_uf,
  doador_partido_sg   as doador_partido,
  doador_candidato_cargo_ds  as doador_cargo,
  doador_genero_ds    as doador_genero,
  doador_label        as doador_label,
  receita_origem_sg   as receita_origem,
  receptor_tipo_cd    as receptor_tipo,
  receptor_uf         as receptor_uf,
  receptor_partido_sg as receptor_partido,
  receptor_cargo_ds   as receptor_cargo,
  receptor_genero_ds  as receptor_genero,
  receptor_label      as receptor_label
FROM {table_receitas}
where
  tabela_id in ('RC','ROP')
group by 
  doador_id           ,
  receptor_id         ,
  doador_tipo_cd      ,
  doador_uf           ,
  doador_partido_sg   ,
  doador_candidato_cargo_ds ,
  doador_genero_ds ,
  doador_label        ,
  receita_origem_sg   ,
  receptor_tipo_cd    ,
  receptor_uf         ,
  receptor_partido_sg ,
  receptor_cargo_ds   ,
  receptor_genero ,
  receptor_label      
;
"""

mtse.execute_query(query_create_table_edges)

## CRIA A TABELA DE NÓS 

In [6]:
query_create_table_nodes = f"""

drop table if exists {table_nodes};
create table {table_nodes}
(
  Id  character varying,
  Label character varying,
  Weight numeric(18,2)
);

CREATE INDEX ON {table_nodes} (Id);

insert into {table_nodes}
select 
     target      , 
     max(receptor_label)
     
FROM {table_edges}  
group by
     target 
;

with nodes_r as 
(select id from {table_nodes} )
insert into {table_nodes} 
select * from
(
select 
     source      , 
     max(doador_label)
    FROM {table_edges}
group by
     source       
) as d
where d.source not in (select id from nodes_r )
;
"""

mtse.execute_query(query_create_table_nodes)

## EXPORTA AS TABELAS DE ARESTAS E NÓS PARA PLANILHAS .CSV

In [7]:
import pandas as pd
df_edges=mtse.pandas_query(f'select * from {table_edges};')
df_edges.to_csv(f'rede{ano_eleicao}_gephi_edges.csv',sep=';',index=False)

df_nodes=mtse.pandas_query(f'select * from {table_nodes};')
df_nodes.to_csv(f'rede{ano_eleicao}_gephi_nodes.csv',sep=';',index=False)